# Домашнее задание 3: Natural Language Inference

In [2]:
# импортируем библиотеки
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/mcdm/CommitmentBank/master/CommitmentBank-items.csv'
df = pd.read_csv(url)

Давайте создадим дополнительную колонку под названием Relation и будем записывать туда entailment, если значение комитмента находится в интервале [1,3]. Если значение комитмента находится в интервале (-1;1), то в колонку Relation записываем neutral. Если же значение комитмента находится в интервале [-3,-1], то в колонку Relation записываем contradiction.

In [9]:
def determine_relation(mean_value):
    if 1 <= mean_value <= 3:
        return 'entailment'
    elif -1 < mean_value < 1:
        return 'neutral'
    elif -3 <= mean_value <= -1:
        return 'contradiction'
    else:
        return 'unknown'

In [10]:
df['Relation'] = df['Mean'].apply(determine_relation)

После того как мы заполнили колонку Relation, давайте создадим колонку label. Если в колонке Relation строка равна entailment, то запишем 0 в колонку label. Если же равна neutral, то запишем 1 в колонку label. Если contradiction, то запишем 2

In [11]:
# Создание колонки "label" на основе значений в колонке "Relation"
df['label'] = df['Relation'].map({'entailment': 0, 'neutral': 1, 'contradiction': 2})
df

,Unnamed: 0,uID,Verb,Embedding,Context,Target,Prompt,ModalType,MatTense,MatSubjLemma,...,Embedding.1,genre,factive,mean.noTarget,sd.noTarget,Mean,SD,Reponses,Relation,label
0,1,BNC-1,admit,conditional,Polly had to think quickly.,They were still close enough to shore for him ...,Polly was not an experienced ocean sailor,NaN,future,she,...,conditional,BNC,no,-0.111111,1.269296,2.000000,0.866025,"2, 3, 3, 1, 3, 1, 2, 1, 2",entailment,0
1,2,BNC-1002,say,modal,"Nevertheless, life went on as it always does. ...",Indeed it could be said that they had prospered.,the Kiwi Keith and the Mackenzie houses had pr...,CI,present,it,...,modal,BNC,no,1.200000,1.873796,1.875000,0.834523,"2, 1, 3, 1, 2, 2, 3, 1",entailment,0
2,3,BNC-1003,say,modal,"He patted her hand. If he had chosen to, Thoma...",He might have said to her that some time in th...,some time in the middle of the nineteenth cent...,AB,present,he,...,modal,BNC,no,-0.250000,0.462910,0.666667,1.154701,"2, 0, 1, 0, 0, 3, -1, 0, 1, 0, 0, 2",neutral,1
3,4,BNC-1005,say,modal,She could see his distorted image in the slant...,Of course she could say it was for the childre...,it was for the children,AB,future,she,...,modal,BNC,no,0.636364,1.120065,0.875000,0.991031,"2, 1, 0, 1, -1, 1, 1, 2",neutral,1
4,5,BNC-1006,say,modal,"She glanced around the room, laying her hot fl...",Robyn swallowed and took a deep breath trying ...,it was all right,AB,future,she,...,modal,BNC,no,-1.090909,1.044466,0.000000,2.309401,"3, 1, -2, -3, -3, 3, 2, 1, -1, -1",neutral,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1196,WSJ-95,feel,negation,"Across the board, these consumers value qualit...",Twenty percent don't even feel they are financ...,they are financially well off,NaN,present,twenty percent,...,negation,WSJ,no,-1.750000,1.035098,-1.000000,2.371708,"-2, 3, -3, -3, -2, -3, -3, 2, -3, -1, 3, -3, 0...",contradiction,2
1196,1197,WSJ-96,mean,negation,The Fed has assumed a similar responsibility i...,This does not mean that the Federal Reserve do...,the Federal Reserve does not already play an i...,NaN,present,this,...,negation,WSJ,no,-0.722222,1.964555,0.125000,2.028957,"2, -2, 3, 0, 2, 1, 2, -2, 2, 1, -1, 0, -2, -3,...",neutral,1
1197,1198,WSJ-97,expect,modal,Under the Racketeer Influenced and Corrupt Org...,"In a letter, prosecutors told Mr. Antar's lawy...",any fees collected from Mr. Antar may be seized,AB,present,they,...,modal,WSJ,no,1.600000,1.264911,2.058824,1.477777,"1, 3, -3, 3, 3, 3, 2, 2, 1, 2, 2, 3, 2, 2, 3, ...",entailment,0
1198,1199,WSJ-98,believe,negation,"Talks between the two sides could unravel, of ...",And although Warner has said it wanted the pro...,the relationship could be repaired after the a...,NaN,present,they,...,negation,WSJ,no,1.083333,1.378954,-1.733333,1.751190,"-1, -3, -3, -1, -2, 1, -3, -3, -3, -3, 2, -2, ...",contradiction,2


Разобьем наш датасет на два набора: набор для изучения(70-80% исходного набора данных) и тестовы набор(20-30% исходного набора данных).

In [12]:
from sklearn. model_selection import train_test_split

train, test = train_test_split(df, test_size= 0.2 , random_state= 0 )

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from nlp import tools
import nlp.tools.tokenization

import os
import numpy as np

In [16]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12"
tokenizer = tools.tokenization.FullTokenizer(vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
     do_lower_case=True)

In [17]:
def encode_sentence(s, tokenizer):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [18]:
def bert_input_encoder(train, tokenizer):
  premises = tf.ragged.constant([encode_sentence(s, tokenizer) for s in np.array(train['Target'])])
  hypotheses = tf.ragged.constant([encode_sentence(s, tokenizer) for s in np.array(train['Prompt'])])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])] * premises.shape[0]

  input_word_ids = tf.concat([cls, premises, hypotheses], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor()

  segment_cls = tf.zeros_like(cls)
  segment_s1 = tf.zeros_like(premises)
  segment_s2 = tf.ones_like(hypotheses)
  input_segment_ids = tf.concat([segment_cls, segment_s1, segment_s2], axis=-1).to_tensor()

  inputs_dic = {
    'input_word_ids': input_word_ids.to_tensor(),
    'input_mask': input_mask,
    'input_segment_ids': input_segment_ids
  }

  return inputs_dic

In [19]:
train_input = bert_input_encoder(train, tokenizer)

train_input

{'input_word_ids': <tf.Tensor: shape=(960, 123), dtype=int32, numpy=
 array([[ 101, 1045, 2123, ...,    0,    0,    0],
        [ 101, 1998, 1045, ...,    0,    0,    0],
        [ 101, 2065, 2012, ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2123, ...,    0,    0,    0],
        [ 101, 2014, 2227, ...,    0,    0,    0],
        [ 101, 1045, 2123, ...,    0,    0,    0]], dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(960, 123), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_segment_ids': <tf.Tensor: shape=(960, 123), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>}

In [21]:
def model_builder():
  max_seq_length = 123

  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
  input_segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_segment_ids")

  bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=True)

  pooled_output, _ = bert_layer([input_word_ids, input_mask, input_segment_ids])

  output_class = tf.keras.layers.Dense(units=3, activation='softmax')(pooled_output)

  model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_segment_ids], outputs=output_class)

  optimizer = tf.keras.optimizers.Adam(lr=1e-5)
  model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

In [22]:
model = model_builder()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 123)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 123)]                0         []                            
                                                                                                  
 input_segment_ids (InputLa  [(None, 123)]                0         []                            
 yer)                                                                                             
                                                                                              

In [22]:
logger = tf.keras.callbacks.TensorBoard(log_dir='/content/drive/My Drive/Collab Notebooks/first_model')

In [ ]:
history = model.fit(
    train_input,
    train.label.values,
    epochs = 2,
    verbose = 1,
    batch_size = 64,
)

Epoch 1/2
